In [97]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding,Dropout,LSTM
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from keras.optimizers import Adam


In [98]:
df=pd.read_csv('stemmedtwitterdataset.csv')

In [99]:
df.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [4]:
df.shape

(1600000, 7)

In [5]:
df.isnull().sum()

target               0
id                   0
date                 0
flag                 0
user                 0
text                 0
stemmed_content    495
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.isnull().sum()

target             0
id                 0
date               0
flag               0
user               0
text               0
stemmed_content    0
dtype: int64

In [8]:
df.shape

(1599505, 7)

In [9]:
tokenizer=Tokenizer(oov_token='none')

In [10]:
tokenizer.fit_on_texts(df['stemmed_content'])

In [11]:

print(tokenizer.word_index)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
print(tokenizer.word_counts)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
print(tokenizer.document_count)

1599505


In [14]:
sequences=tokenizer.texts_to_sequences(df['stemmed_content'])
sequences

[[15722, 10, 82, 22, 16569, 371, 1060, 3131, 13, 639, 7331, 1707, 4],
 [617, 227, 416, 366, 212, 238, 994, 79, 11, 195, 1016],
 [22100, 3715, 234, 12, 826, 720, 518, 357, 2, 2756],
 [349, 661, 25, 2589, 7, 859],
 [28430, 4141, 463, 24],
 [193578, 349, 1918],
 [33, 397],
 [118586, 88, 101, 12, 24, 89, 102, 66, 66, 14, 436, 15],
 [14397, 341, 668],
 [56982, 2037, 193579],
 [1301, 334, 2715, 482, 1433],
 [2447, 784],
 [38888, 1272, 31, 153, 8524, 1404, 2869],
 [193580, 781, 716, 389, 97, 126, 318],
 [193581, 59, 106, 2317, 27, 69, 3293, 28431, 193582],
 [193583, 46, 13, 31, 17, 193584, 2148],
 [2192, 931, 1586, 141, 1509, 31, 647, 20435, 4435, 456],
 [1195, 2336],
 [68342, 584, 552, 125, 20, 24, 1560, 8, 3262],
 [118587, 38, 516, 276, 2389, 1477, 276],
 [193585, 4, 3, 41, 119],
 [18, 63, 116, 296, 221, 2205, 3183, 11, 12, 494],
 [1517, 12081, 1282, 637, 580],
 [53, 2, 30],
 [38889, 283, 45],
 [73, 116, 319, 80],
 [2, 238, 49, 31, 2920],
 [23, 51, 17, 5720],
 [2846, 14, 5480, 114, 5480, 3,

In [15]:
longest_train_sequence=len(max(sequences,key=len))
longest_train_sequence

50

In [16]:
padded_sequence=pad_sequences(sequences,maxlen=longest_train_sequence,padding='post')
padded_sequence

array([[ 15722,     10,     82, ...,      0,      0,      0],
       [   617,    227,    416, ...,      0,      0,      0],
       [ 22100,   3715,    234, ...,      0,      0,      0],
       ...,
       [   142,   5633,   6498, ...,      0,      0,      0],
       [    57,    200,    190, ...,      0,      0,      0],
       [    57,  15640, 191224, ...,      0,      0,      0]])

In [17]:
padded_sequence.shape

(1599505, 50)

In [18]:
X_train,X_test,y_train,y_test=train_test_split(padded_sequence,df['target'],test_size=0.2,stratify=df['target'],random_state=2)

In [19]:
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("t_test",y_test.shape)

X_train (1279604, 50)
X_test (319901, 50)
y_train (1279604,)
t_test (319901,)


In [20]:
vocab_size=max([max(x) for x in X_train])+1
vocab_size

532722

In [100]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.regularizers import l2


In [101]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5,  verbose=1,mode='min')

In [102]:
checkpoint = ModelCheckpoint('model.h5', save_best_only=True, verbose=1,monitor='val_loss', mode='min')

In [112]:
model=Sequential()
model.add(Embedding(vocab_size,output_dim=32,input_length=longest_train_sequence))
model.add(Dropout(0.5))
model.add(LSTM(20, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[532722,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
%%time
# history=model.fit(X_train,y_train,epochs=5,batch_size=64,verbose=1,validation_data=(X_test,y_test))
history=model.fit(X_train,y_train,epochs=8,batch_size=1024,validation_data=(X_test,y_test),  verbose=1,callbacks=[checkpoint,early_stopping])


In [25]:
import pickle

In [26]:
with open('rnn_twitter_sentiment_analysis_modal_87acc.pkl','wb') as f:
    pickle.dump(model,f)

INFO:tensorflow:Assets written to: ram://8d882f89-c40c-4735-82a2-b6db1ebc3041/assets


In [ ]:
with open('rnn_twitter_sentiment_analysis_tokenizer_87acc.pkl','wb') as fo:
    pickle.dump(tokenizer,fo)

In [27]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [28]:
X_train_predict = model.predict(X_train)
X_train_predict = [1 if prob > 0.5 else 0 for prob in X_train_predict]

 2146/39988 [>.............................] - ETA: 6:34

KeyboardInterrupt: 

In [52]:
#from tensorflow.keras.models import load_model

# Load the model
#model = load_model('model.h5')

# Continue training
#history=model.fit(X_train,y_train,epochs=15,validation_data=(X_test,y_test), callbacks=[checkpoint])

In [ ]:
#model.save('my_model.h5')
##save tbe model

In [ ]:
training_data_acc = accuracy_score(y_train, X_train_predict)

In [ ]:
y_pred = model.predict(X_test)
y_pred_bin = [1 if prob > 0.5 else 0 for prob in y_pred]

In [ ]:
test_data_acc = accuracy_score(y_test, y_pred_bin)

In [ ]:
if training_data_acc > test_data_acc:
    print("The model might be overfitting")
else:
    print("The model is not overfitting")

In [ ]:
print(classification_report(y_test, y_pred_bin))

In [ ]:
print(confusion_matrix(y_test, y_pred_bin))

In [ ]:
print("Train data accuracy is: ",training_data_acc)
print("Test data accuracy is: ",test_data_acc)

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [41]:
import pickle
from tensorflow.keras.models import load_model

In [45]:


# Load the model from the .pkl file
with open('twittersentiment.pkl', 'rb') as file:
    model = pickle.load(file)



FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://c428ec94-7df1-4aa6-81e8-d7a29a2140e0/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [36]:
# Save the model in .h5 format
model.save('twittersentiment.h5')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
